# Machine learning algorithm part

Hello and wellcome to machine learning algorithm notebook! In this notebook we gonna to build, train & evaluate machine learning algorithm using dimensionality reduction technique, data augmentation and underfitting ideas from previous notebook. First of all, let's import all necessary libraries:

In [1]:
import matplotlib as plt                                 
import pandas as pd                                      
import numpy as np                                   

import io                                                # operations with file
import socket                                            # Socket Server

from sklearn.neural_network import MLPClassifier         # Neural Network
from sklearn.metrics import classification_report        # Classification metrics
from sklearn.preprocessing import normalize              # Data normalization
from sklearn.model_selection import train_test_split

from PIL import Image                                    # operations with images

%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Users\User\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


## Data dowloading
Now we gonna to download the "labels.csv" file with names of all images and their labels by specific path (path in your case can be different):

In [2]:
pathToImages = 'C:/Users/User/Desktop/Polygon/Dataset/'
dataTable = pd.read_csv(pathToImages + 'labels.csv', header = 0, index_col = None)
train = []
labels = []

# Labels: 1 - forward; 2 - rightward; 3 - stop; 4 - leftward;

dataTable.head()

,Image name,Direction
0,Lidar_2_16_2018 5_00_18 PM_66.jpg,1
1,Lidar_2_16_2018 5_00_18 PM_123.jpg,1
2,Lidar_2_16_2018 5_00_18 PM_184.jpg,1
3,Lidar_2_16_2018 5_00_18 PM_235.jpg,1
4,Lidar_2_16_2018 5_00_18 PM_284.jpg,1


## Dimensionality reduction
Also we will download all the images and apply dimensionality reduction technique:

In [3]:
%%time
# Convert images with size 180x64 to vectors of length 180
for filename in dataTable['Image name']:
    imagePixels = []
    image = Image.open((pathToImages + 'images/' +  filename)) #Can be many different formats.
    pixels = list(image.getdata())
    for i in range(image.size[0]):
        imagePixels.append(np.mean(pixels[i::180]))
    train.append(imagePixels)
        
# Save the labels of the images
labels = list(dataTable['Direction'])

Wall time: 3min 25s


## Data augmentation

Now we need to generate new data from the existing dataset, to decrease imbalance in data content. Also don't forget to normalize data and split for train/test sets.

In [4]:
# Synthesizing data: vertical inverting of images
for i in range(len(dataTable['Direction'])):
    train.append(train[i][::-1])
    if labels[i] == 2:
        labels.append(4)
    elif labels[i] == 4:
        labels.append(2)
    else:
        labels.append(1)

In [8]:
# Normalize data
normal = normalize(train, norm='max')

X_train, X_test, y_train, y_test = train_test_split(normal, labels, test_size=0.1, random_state=42)

print("Total images: " + str(len(train)))

print("\nTrain example: ")
print(np.around(train[0], decimals = 7))
print("\nNormalized example: ")
print(normal[0])

Total images: 26880

Train example: 
[ 31.703125   31.703125   31.703125   31.703125   31.703125   31.703125
  31.703125   31.703125   31.703125   31.7291667  31.7916667  32.390625
  32.46875    33.1145833  33.21875    33.265625   32.7916667  32.828125
  32.9166667  33.5572917  33.4375     33.9270833  34.         34.015625
  34.609375   34.6510417  34.84375    35.203125   35.3802083  35.9635417
  36.2291667  36.265625   36.7552083  36.7708333  37.3177083  37.9583333
  38.09375    38.5989583  39.078125   39.046875   40.0989583  40.1614583
  40.5885417  41.0989583  42.0677083  42.6875     43.3645833  43.4791667
  44.3177083  44.4791667  45.203125   45.7708333  46.7864583  47.3385417
  47.7760417  48.0833333  49.5677083  49.984375   50.765625   51.7083333
  52.4166667  53.53125    54.296875   54.84375    56.0572917  56.7447917
  58.1822917  58.984375   59.6822917  60.8229167  61.7291667  62.4583333
  63.515625   64.265625   65.1302083  66.0885417  66.6614583  66.859375
  66.7135417  66.61

## Model building

As model we will use neural network. Sklearn already has realisation of multilayer perceptron, so we will apply it. Remember that we need to underfit our model a little bit to make it more adequate! So in our case approximately ~90% accuracy on train/test sets should be enough.

In [9]:
classifier = MLPClassifier(hidden_layer_sizes=(20), max_iter=2000, activation='logistic', learning_rate_init=0.00001, 
                           learning_rate='adaptive', random_state=42, shuffle=True)

In [10]:
%%time
classifier.fit(X_train, y_train)

Wall time: 2min 5s


MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=20, learning_rate='adaptive',
       learning_rate_init=1e-05, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [11]:
print("Train:")
print(classification_report(y_train, classifier.predict(X_train)))
print("Test:")
print(classification_report(y_test, classifier.predict(X_test)))

Train:
             precision    recall  f1-score   support

          1       0.94      0.96      0.95     16425
          2       0.92      0.86      0.89      3870
          4       0.88      0.88      0.88      3897

avg / total       0.93      0.93      0.93     24192

Test:
             precision    recall  f1-score   support

          1       0.94      0.95      0.95      1817
          2       0.94      0.85      0.89       449
          4       0.86      0.91      0.88       422

avg / total       0.93      0.93      0.93      2688



## Environment & Agent interaction
This class provides the interaction between environment and agent, or model in other words. The class send the specific command from "actionList" and receive the lidar image from Unity. <b>Note: <i>The class work as server. So you should run this class first and then run Unity part. Because server will wait connection from Unity part.</i></b>

In [10]:
# Declaring a class

class EnvironmentInteraction:
    
    def __init__(self, Address, Port):
        
        self.address = Address
        self.port = Port
        self.sock = socket.socket()
        self.sock.bind((self.address, self.port))
            
        
    def listen(self):
        
        self.sock.listen(1)
        self.connection, self.address = self.sock.accept()
        
        
    def step(self, action):
        
        imagePixels = []
        
        self.connection.send(action.encode('utf-8'))                        # Send action to environment
        data = self.connection.recv(128000)                                 # Get lidar image as bytes data 
        image = Image.frombytes('RGB', (180, 64), data)
        
        pixels = list(image.getdata())
    
        for i in range(image.size[0]):
            imagePixels.append(np.mean(pixels[i::180]))
                
        return imagePixels
    
environment = EnvironmentInteraction('', 7777)        

In [49]:
# Model execution

environment.listen()
actionList = ['forward', 'rightward', 'backward', 'leftward']
print(environment.connection)

outputLayer = 'forward'

while True:
        
    inputLayer = environment.step(outputLayer)    
    inputLayer = normalize([inputLayer], norm='max')
    
    outputLayer = actionList[int(classifier.predict(inputLayer)[0] - 1)]
#     print(outputLayer)

<socket.socket fd=2076, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 7777), raddr=('127.0.0.1', 49969)>


ConnectionResetError: [WinError 10054] Удаленный хост принудительно разорвал существующее подключение